# Decision Tree Data Outcomes

This module requires the completion of [the initial data loading module](1%20-%20Loading%20Decision%20Tree%20Data.ipynb).  Please complete this module before continuing.

As stated in the previous modules, the decision tree data focuses around determining what drink we want at starbucks.  The decision tree assists users by narrowing down the amount of answers required to describe a regulatory document.  How can we make use of this decision tree?  This module connects answers together with `Set` nodes.  These represent groups of valid responses for a particular document.

In [2]:
# Import os utilities
import os
from string import Template
import pandas as pd

# Import Python 2 Neo4J Package
from py2neo import Graph

# Connect to our Graph database, ensure connectivity, and store connection in variable.
graph = Graph("bolt://localhost", auth=("neo4j", "123changeme"));

# Set up a local path reference
rel_path = os.getcwd()

The `sets.csv` file contains mock data, but the concepts remains the same.  We should be able to determine what drinks apply based on user input in the form of answers.

In [3]:
# This file contains scans our tool ran on presidential candidates.
sets_file = "file:" + os.path.join(rel_path, "sets.csv");

In [4]:
import_query = """
LOAD CSV WITH HEADERS FROM $setsFile AS set
WITH set WHERE set.id IS NOT NULL AND set.answer IS NOT NULL
MERGE (s:Set { id: toInt(set.id) })
SET s.value = set.name
WITH s, set
MATCH (a:Answer { id: toInt(set.answer) })
CREATE UNIQUE (s)-[:GROUPS]->(a)
"""

display(graph.run("CREATE CONSTRAINT ON (s:Set) ASSERT s.id IS UNIQUE").summary().counters)
display(graph.run(import_query, { "setsFile": sets_file }).summary().counters)

{}

{'labels_added': 5, 'relationships_created': 27, 'nodes_created': 5, 'properties_set': 32}

## Data Loaded

Now we have a handful of sets created that are connected to multiple answers.  We will be able to map a group of answers to all possible resulting sets.

&nbsp;

<div align="left">
    <img src="images/set_groups.png" alt="Graph Model" width="700px" align="center"/>
</div>

&nbsp;

Let's check out what sets were created in the database.  The following query gives us all nodes with the label `Set` in the database.

In [6]:
result = graph.run("MATCH (s:Set) RETURN s.id AS ID, s.value AS Value");
table = result.to_data_frame(columns=['ID', 'Value']);

display(table);

,ID,Value
0,1,Tall Iced Coffee
1,2,Grande Java Chip Frappucino
2,3,Venti Caremet Macchiato
3,4,Tall Fuzzy Peach Tea
4,5,Grande Pumpkin Spice Frappucino


It looks like there are only five `Sets` defined in the data.  The following query looks at the relationship between a `Set` and `Answers`.

In [10]:
result = graph.run("""
MATCH (s:Set { id: 1 })-->(a)<--(q)
WHERE 'Question' IN labels(q)
RETURN s.value AS Set, q.value as Question, a.value as Answer, a.id as Answer_ID
""");
table = result.to_data_frame(columns=['Set', 'Question', 'Answer', 'Answer_ID']);

display(table);

,Set,Question,Answer,Answer_ID
0,Tall Iced Coffee,Size,Tall,11
1,Tall Iced Coffee,Is Holiday,No,2
2,Tall Iced Coffee,Is Espresso Based,No,21
3,Tall Iced Coffee,Is Iced,Yes,18
4,Tall Iced Coffee,Is Frappucino,No,4
5,Tall Iced Coffee,Is Tea,No,29


## Data Exploration

Now we want to see how we can take an answer set and relate it back to our documents within the `Set` nodes.  The following query takes a group of answers and determines if a given `Set` is valid.

In [11]:
def is_set_valid(set_id, answer_ids):
    query_string = """
        MATCH (s:Set { id: $set_id })-[:GROUPS]->(a:Answer)
        WHERE a.id IN $answer_ids
        WITH collect(a) as answers
        RETURN size(answers) = size($answer_ids) as Result
    """
    template = Template(query_string);
    query = template.substitute(set_id=set_id, answer_ids=answer_ids);
    return graph.run(query);
    
MY_SET_ID = 5; # Grande Pumpkin Spice Frappucino
MY_VALID_ANSWER_IDS = [1, 6, 12]; # you can change this to any list to test other answer ids
# Is Holiday: Yes, Frappucino Flavor: Pumpkin Spice, Size: Grande
MY_INVALID_ANSWER_IDS = [2, 6, 12]; # you can change this to any list to test other answer ids
# Is Holiday: *NO*, Frappucino Flavor: Pumpkin Spice, Size: Grande

valid_result = is_set_valid(MY_SET_ID, MY_VALID_ANSWER_IDS);
invalid_result = is_set_valid(MY_SET_ID, MY_INVALID_ANSWER_IDS);

print("Valid ID List:");
display(valid_result.to_data_frame());

print("Invalid ID List:");
display(invalid_result.to_data_frame());

Valid ID List:


,Result
0,True


Invalid ID List:


,Result
0,False


This demonstrates the fact you can't get a Pimpkin Spice Frappucino outside of holiday periods.

### The concluding query.

The most important question we want to ask our data is: _What are all valid `Set` nodes for a given list of answers?_  The following code will give us a list of available drinks by analyzing a list of answers in the decision tree.

In [45]:
def get_answers(answer_ids):
    query_string = """
        MATCH (a:Answer)<--(q:Question)
        WHERE a.id IN $answer_ids
        RETURN a.id AS ID, q.value as Question, a.value AS Value
    """
    template = Template(query_string);
    query = template.substitute(answer_ids=answer_ids);
    return graph.run(query).to_data_frame();

def match_sets(answer_ids):
    query_string = """
        MATCH (s:Set)-[:GROUPS]->(a:Answer)
        WHERE a.id IN $answer_ids
        WITH s, collect(a) as answers
        WHERE size(answers) = size($answer_ids)
        RETURN s
    """
    template = Template(query_string);
    query = template.substitute(answer_ids=answer_ids);
    return graph.run(query);
    
EXAMPLE_A_ANSWERS = [2, 4, 11];
EXAMPLE_B_ANSWERS = [2, 4, 11, 28];
a_result_list = None;
b_result_list = None;

a_result = match_sets(EXAMPLE_A_ANSWERS);
b_result = match_sets(EXAMPLE_B_ANSWERS);

print("A Answers:");
display(get_answers(EXAMPLE_A_ANSWERS));

print("A Drinks:");
a_nodes = a_result.to_subgraph();
if a_nodes is not None:
    a_result_list = list(a_nodes.nodes);
    a_result_list = list(map(lambda n: [n['id'], n['value']], a_result_list));

display(a_result_list or []);
print("");
print("B Answers:");
display(get_answers(EXAMPLE_B_ANSWERS));

print("B Drinks:");
b_nodes = b_result.to_subgraph();
if b_nodes is not None:
    b_result_list = list(b_nodes.nodes);
    b_result_list = list(map(lambda n: [n['id'], n['value']], b_result_list));

display(b_result_list or []);

A Answers:


,ID,Question,Value
0,4,Is Frappucino,No
1,2,Is Holiday,No
2,11,Size,Tall


A Drinks:


[[1, 'Tall Iced Coffee'], [4, 'Tall Fuzzy Peach Tea']]


B Answers:


,ID,Question,Value
0,2,Is Holiday,No
1,11,Size,Tall
2,4,Is Frappucino,No
3,28,Is Tea,Yes


B Drinks:


[[4, 'Tall Fuzzy Peach Tea']]